In [ ]:
import os
import math
import json
import random
import datetime
import numpy as np
import pandas as pd
import time

import seaborn as sns
import matplotlib.pyplot as plt

# Signal Processing
import scipy.signal as sig
import scipy.stats as stats

# Sklearn
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier, RandomForestRegressor
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline
import lightgbm as lgb

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def drop_correlation(df, labels, threshold = 0.95, plotcorr = False):
    corr = df.loc[:, ~df.columns.isin(labels)].corr()
    if plotcorr: 
        f, ax = plt.subplots(figsize=(15, 15))
        cmap = sns.diverging_palette(220, 10, as_cmap=True)
        sns.heatmap(corr, cmap = cmap,
                xticklabels=corr.columns.values,
                yticklabels=corr.columns.values)
    # Select upper triangle of correlation matrix
    upper = corr.abs().where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    # Find features with correlation greater than threshold
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    print("Columns dropped: ", len(to_drop))
    # Drop features 
    df.drop(columns = to_drop, inplace=True)
    print("New Dataframe Shape: " + str(df.shape))
    return(df)

def predict_bp_from_ppg(dataframe, predicted_variable = 'SBP', k = 1, 
                        correlation_threshold = 0.95,
                        random_seed = 42, learning_rate = 0.01, 
                        n_estimators = 100, 
                        alpha = 1, l1_ratio = 0.5, random_state = 42, 
                        epochs = 50, 
                        batch_size = 5, n_jobs = -1, max_depth = 10, 
                        verbose = False):    
        
    df = dataframe.rename(columns={"SYS(mmHg)": "SBP", "DIA(mmHg)": "DBP", 'subject': 'patientid'})
    
    # Dropping correlation
    df.drop(df.loc[(df['SBP'] == 0)|(df['DBP'] == 0)].index, inplace = True)
    df = drop_correlation(df, ['SBP', 'DBP'], correlation_threshold, plotcorr = False)
    if verbose: print(df.shape)

    features = df.shape[1]-3
    print('Nr of features: ', features)
    patient_ids = np.unique(df['patientid'])
    
    # Create estimators
    estimators_lr = []
    estimators_gbm = []
    estimators_lgbm = []
    estimators_rf = []

    estimators_lr.append(('standardize', StandardScaler()))
    estimators_lr.append(('lr', ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=random_state)))

    estimators_gbm.append(('standardize', StandardScaler()))
    estimators_gbm.append(('gbm', GradientBoostingRegressor(learning_rate=learning_rate, n_estimators=n_estimators, random_state=random_seed)))
    
    estimators_lgbm.append(('standardize', StandardScaler()))
    estimators_lgbm.append(('lgbm', lgb.LGBMRegressor(learning_rate=learning_rate, n_estimators=n_estimators, random_state=random_seed, n_jobs=n_jobs)))
    

    estimators_rf.append(('standardize', StandardScaler()))
    estimators_rf.append(('rf', RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state, n_jobs=n_jobs)))
            
    pipeline_lr = Pipeline(estimators_lr)
    pipeline_gbm = Pipeline(estimators_gbm)
    pipeline_lgbm = Pipeline(estimators_lgbm)
    pipeline_rf = Pipeline(estimators_rf)

    RMSE_LR = []
    MAPE_LR = []
    MAE_LR = []

    RMSE_GBM = []
    MAPE_GBM = []
    MAE_GBM = []
    
    RMSE_DUMMY = []
    MAPE_DUMMY = []
    MAE_DUMMY = []
    
    RMSE_LGBM = []
    MAPE_LGBM = []
    MAE_LGBM = []
    
    RMSE_RF = []
    MAPE_RF = []
    MAE_RF = []

    results = {}
    i = 0
    mean_train = 0
    mean_test = 0
    total = len(df.index)
    subjects = len(df['patientid'].unique())
    
    if verbose: print("BPPairs: ", total)
    if verbose: print("Subjects: ", subjects)
    if verbose: print("\n")

    while len(patient_ids) > 1:
        i= i + 1 

        # Random Seed
        random.seed(random_seed)
        
        # Drop Condition
        drop_condition = 'any'

        patient_test_ids = random.choices(patient_ids, k = k)
        patient_ids = [e for e in patient_ids if e not in patient_test_ids]
        df_test = df.loc[df['patientid'].isin(patient_test_ids)].dropna(how=drop_condition)
        df_train = df[~df['patientid'].isin(patient_test_ids)].dropna(how=drop_condition)
        if verbose: print("Running fold" + str(i))
        if verbose: print("Train: ", df_train.shape)
        mean_train += len(df_train.index)
        if verbose: print("Test: ", df_test.shape)
        if verbose: print("Total: ", len(df_test.index) + len(df_train.index))
        mean_test += len(df_test.index)
        if verbose: print("\n")

        cols_dropped = ['patientid']

        if predicted_variable == 'SBP':
            cols_dropped.append('DBP')
        elif predicted_variable == 'DBP':
            cols_dropped.append('SBP')
        df_train = df_train.drop(columns = cols_dropped)
        df_test = df_test.drop(columns = cols_dropped)

        #lr
        pipeline_lr.fit(X = df_train.loc[:, df_train.columns != predicted_variable].values, 
                        y = df_train[predicted_variable].values)
        predicted_labels = pipeline_lr.predict(df_test.loc[:, df_test.columns != predicted_variable].values)

        RMSE_LR.append(np.sqrt(mean_squared_error(df_test[predicted_variable], predicted_labels)))  
        MAPE_LR.append(mean_absolute_percentage_error(df_test[predicted_variable], predicted_labels))
        MAE_LR.append(mean_absolute_error(df_test[predicted_variable], predicted_labels))

        #gbm 
        pipeline_gbm.fit(X = df_train.loc[:, df_train.columns != predicted_variable].values, 
                         y = df_train[predicted_variable].values)
        predicted_labels = pipeline_gbm.predict(df_test.loc[:, df_test.columns != predicted_variable].values)

        RMSE_GBM.append(np.sqrt(mean_squared_error(df_test[predicted_variable], predicted_labels)))  
        MAPE_GBM.append(mean_absolute_percentage_error(df_test[predicted_variable], predicted_labels))
        MAE_GBM.append(mean_absolute_error(df_test[predicted_variable], predicted_labels))
        
        #lightgbm
        pipeline_lgbm.fit(X = df_train.loc[:, df_train.columns != predicted_variable].values, y = df_train[predicted_variable].values)
        predicted_labels = pipeline_lgbm.predict(df_test.loc[:, df_test.columns != predicted_variable].values)

        RMSE_LGBM.append(np.sqrt(mean_squared_error(df_test[predicted_variable], predicted_labels)))  
        MAPE_LGBM.append(mean_absolute_percentage_error(df_test[predicted_variable], predicted_labels))
        MAE_LGBM.append(mean_absolute_error(df_test[predicted_variable], predicted_labels))
        
        #rf
        pipeline_rf.fit(X = df_train.loc[:, df_train.columns != predicted_variable].values, y = df_train[predicted_variable].values)
        predicted_labels = pipeline_rf.predict(df_test.loc[:, df_test.columns != predicted_variable].values)

        RMSE_RF.append(np.sqrt(mean_squared_error(df_test[predicted_variable], predicted_labels)))  
        MAPE_RF.append(mean_absolute_percentage_error(df_test[predicted_variable], predicted_labels))
        MAE_RF.append(mean_absolute_error(df_test[predicted_variable], predicted_labels))
        
        # dummy_mean
        X = df_train.loc[:, df_train.columns != predicted_variable].values
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        dummy_mean = DummyRegressor(strategy='mean')
        
        dummy_mean.fit(X_scaled, 
                         y = df_train[predicted_variable].values)
        predicted_labels = dummy_mean.predict(df_test.loc[:, df_test.columns != predicted_variable].values)

        RMSE_DUMMY.append(np.sqrt(mean_squared_error(df_test[predicted_variable], predicted_labels)))  
        MAPE_DUMMY.append(mean_absolute_percentage_error(df_test[predicted_variable], predicted_labels))
        MAE_DUMMY.append(mean_absolute_error(df_test[predicted_variable], predicted_labels))
    
    # General Info
    results['subjects'] = subjects
    results['bp_pairs'] = total
    results['folders'] = i
    results['mean_train_size'] = round(mean_train/i)
    results['mean_test_size'] = round(mean_test/i)
    
    # Mean LR
    results['RMSE_LR_MEAN'] = np.mean(np.array(RMSE_LR))
    results['MAPE_LR_MEAN'] = np.mean(np.array(MAPE_LR))
    results['MAE_LR_MEAN'] = np.mean(np.array(MAE_LR))
            
    # STD LR
    results['RMSE_LR_STD'] = np.std(np.array(RMSE_LR))
    results['MAPE_LR_STD'] = np.std(np.array(MAPE_LR))
    results['MAE_LR_STD'] = np.std(np.array(MAE_LR))

    # Mean GBM
    results['RMSE_GBM_MEAN'] = np.mean(np.array(RMSE_GBM))
    results['MAPE_GBM_MEAN'] = np.mean(np.array(MAPE_GBM))
    results['MAE_GBM_MEAN'] = np.mean(np.array(MAE_GBM))
    
    # Std GBM
    results['RMSE_GBM_STD'] = np.std(np.array(RMSE_GBM))
    results['MAPE_GBM_STD'] = np.std(np.array(MAPE_GBM))
    results['MAE_GBM_STD'] = np.std(np.array(MAE_GBM))
    
    # Mean LGBM
    results['RMSE_LGBM_MEAN'] = np.mean(np.array(RMSE_LGBM))
    results['MAPE_LGBM_MEAN'] = np.mean(np.array(MAPE_LGBM))
    results['MAE_LGBM_MEAN'] = np.mean(np.array(MAE_LGBM))
    
    # Std LGBM
    results['RMSE_LGBM_STD'] = np.std(np.array(RMSE_LGBM))
    results['MAPE_LGBM_STD'] = np.std(np.array(MAPE_LGBM))
    results['MAE_LGBM_STD'] = np.std(np.array(MAE_LGBM))
    
    # Mean RF
    results['RMSE_RF_MEAN'] = np.mean(np.array(RMSE_RF))
    results['MAPE_RF_MEAN'] = np.mean(np.array(MAPE_RF))
    results['MAE_RF_MEAN'] = np.mean(np.array(MAE_RF))
    
    # Std RF
    results['RMSE_RF_STD'] = np.std(np.array(RMSE_RF))
    results['MAPE_RF_STD'] = np.std(np.array(MAPE_RF))
    results['MAE_RF_STD'] = np.std(np.array(MAE_RF))
    
    # Mean Dummy
    results['RMSE_DUMMY_MEAN'] = np.mean(np.array(RMSE_DUMMY))
    results['MAPE_DUMMY_MEAN'] = np.mean(np.array(MAPE_DUMMY))
    results['MAE_DUMMY_MEAN'] = np.mean(np.array(MAE_DUMMY))
    
    # Std Dummy
    results['RMSE_DUMMY_STD'] = np.std(np.array(RMSE_DUMMY))
    results['MAPE_DUMMY_STD'] = np.std(np.array(MAPE_DUMMY))
    results['MAE_DUMMY_STD'] = np.std(np.array(MAE_DUMMY))
    
    parameters = {
                    'predicted_variable' : predicted_variable,
                    'correlation_threshold' : correlation_threshold, 
                    'random_seed' :  random_seed,
                    'learning_rate' : learning_rate, 
                    'n_estimators' : n_estimators, 
                    'alpha' : alpha, 
                    'l1_ratio' : l1_ratio,
                    'random_state' : random_state, 
                    'k' : k, 
                    'features' : features, 
                    'epochs' : epochs, 
                    'batch_size' : batch_size,
                    'max_depth' : max_depth,
                    'n_jobs' : n_jobs,
    }    
    results.update(parameters)
    return results

In [ ]:
def get_files(myPath):
    file_names = [f for f in os.listdir(myPath) if os.path.isfile(os.path.join(myPath, f)) and f[-4:] == ".txt"]
    file_names.sort()
    return file_names

def apply_filter(df, filter_type='cheby', verbose=False):
    if filter_type == 'avg':
        df['bvp_filtered'] = pd.DataFrame(np.convolve(df['bvp'], np.ones((100,))/100, mode='valid'))
    elif filter_type == 'cheby':
        sos = sig.cheby2(4, 20, [0.5, 8], btype='bandpass', fs=1000, output='sos')
        df['bvp_filtered'] = sig.sosfiltfilt(sos, df['bvp'])
    elif filter_type == 'butter':
        sos = sig.butter(4, [0.5, 8], btype='bandpass', fs=1000, output='sos')
        df['bvp_filtered'] = sig.sosfiltfilt(sos, df['bvp'])
    if verbose:
        fig = plt.figure(figsize=(14, 6))
        plt.plot(df.index, df['bvp_filtered'])
        plt.xlabel('Samples')
        plt.ylabel('Scaled Magnitude')
        
        # Save PPG Filtered
        with open('../../config.json') as f:
            config = json.load(f)
        today = datetime.datetime.today().strftime('%Y-%m-%d')
        figure_path = config['figures']
        
        ppg_figure_path = os.path.join(figure_path, today, 'ppg_liang')
        if not os.path.exists(ppg_figure_path):
            os.makedirs(ppg_figure_path)
        
        fig.savefig(os.path.join(ppg_figure_path, 'filter_liang.pdf'), dpi=300, format='pdf')
        plt.show()
    return df

def normalise(df, method='z_score'):
    if 'min_max':
        df['bvp'] = (df['bvp'] - df['bvp'].min()) / (df['bvp'].max() - df['bvp'].min())
    elif 'z_score':
        df['bvp'] =  (df['bvp'] - df['bvp'].mean()) / df['bvp'].std()
    return df

# finds the local minima that correspond to the starts of a cardiac cycle
def find_cycle_starts(df, sample_rate=1000):
    minima = sig.find_peaks(-df.values, distance=0.7*sample_rate)[0]
    return minima

# returns the x values for those samples in the signal, that are closest to some given y value
def find_xs_for_y(ys, y_val, sys_peak):
    diffs = abs(ys - y_val)
    x1 = diffs[:sys_peak].idxmin()
    x2 = diffs[sys_peak:].idxmin()
    return x1, x2

# takes a dataframe of calculated features and removes the outliers occurring due to inaccuracies in the signal
def clean_window_features_of_outliers(df):
    quant = df.quantile(0.8)
    for col in df.columns:
        if col.find('ts') == -1:
            df = df[df[col] < quant[col]*2]
    return df

def find_clean_cycles_with_template(signal, verbose=False):
    initial_cycle_starts = find_cycle_starts(signal)
    if len(initial_cycle_starts) <= 1:
        return []
    template_length = math.floor(np.median(np.diff(initial_cycle_starts)))
    cycle_starts = initial_cycle_starts[:-1]
    while cycle_starts[-1] + template_length > len(signal):
        cycle_starts = cycle_starts[:-1]
    template = []
    for i in range(template_length):
        template.append(np.mean(signal[cycle_starts + i]))
    
    corr_coef = []
    for cycle_start in cycle_starts:
        corr_coef.append(np.corrcoef(template, signal[cycle_start:cycle_start+template_length])[0,1])

    valid_indices = np.argwhere(np.array(corr_coef) >= 0.8)
    if (len(valid_indices) > len(cycle_starts) / 2) and len(valid_indices) > 1:
        cycle_starts = cycle_starts[np.squeeze(valid_indices)]
        template2 = []
        for i in range(template_length):
            template2.append(np.mean(signal[cycle_starts + i]))
        template = template2
        
    if verbose:
        print('Cycle Template')
        plot = plt.plot(template)
        plt.show()
        
    # Check correlation of cycles with template
    # SQI1: Pearson Correlation
    sqi1_corr = []
    for cycle_start in cycle_starts:
        corr, _ = stats.pearsonr(template, np.nan_to_num(signal[cycle_start:cycle_start+template_length]))
        sqi1_corr.append(corr)
        
    # SQI2: Pearson Correlation with 
    sqi2_corr = []
    for cycle_start in cycle_starts:
        cycle_end = initial_cycle_starts[np.squeeze(np.argwhere(initial_cycle_starts==cycle_start)) + 1] 
        corr, _ = stats.pearsonr(template, sig.resample(signal[cycle_start:cycle_end], template_length))
        sqi2_corr.append(corr)
        
    # SQI3: Pearson Correlation
    corrs = np.array([sqi1_corr, sqi2_corr]).transpose()
    cycle_starts = cycle_starts[np.all(corrs >= 0.8, axis=1)]
    
    if verbose:
        print('Detected Valid Cycles')
        fig = plt.figure(figsize=(12, 10), dpi=300)
        for cycle_start in cycle_starts:
            plt.rcParams.update({'font.size': 16})
            plt.plot(signal[cycle_start:cycle_start+template_length].to_numpy())
        
        # Save Valid Cycles
        with open('../../config.json') as f:
            config = json.load(f)
        today = datetime.datetime.today().strftime('%Y-%m-%d')
        figure_path = config['figures']
    
        millis = int(round(time.time() * 1000))
        valid_cycles = os.path.join(figure_path, today, 'valid_cycles_liang')
        
        if not os.path.exists(valid_cycles):
            os.makedirs(valid_cycles)
        fig.savefig(os.path.join(valid_cycles, str(millis)+'_valid_cycles_liang.png'))
        
    cycles = []
    for cycle_start in cycle_starts:
        cycle_end = initial_cycle_starts[np.squeeze(np.argwhere(initial_cycle_starts==cycle_start)) + 1]
        if (cycle_end - cycle_start) > template_length*1.2:
            cycle_end = cycle_start + template_length
        cycles.append((cycle_start, cycle_end))

    return cycles
    
# Filter PPG Data
def extract_features_for_cycle(window_df, signal, verbose=False):
    cur_index = window_df.index.max() + 1
    if np.isnan(cur_index):
        cur_index = 0
    signal.index = pd.to_datetime(signal.index, unit='ms')
    signal = signal.interpolate(method='time')
    signal = signal - signal.min()
    max_amplitude = signal.max()
    
    peaks = sig.find_peaks(signal.values, distance=0.7*1000)[0]
    sys_peak_ts = signal.index[peaks[0]]
    
    if verbose:
        # Save Features in Cycle
        with open('../../config.json') as f:
            config = json.load(f)
        today = datetime.datetime.today().strftime('%Y-%m-%d')
        figure_path = config['figures']
        
        features_cycle = os.path.join(figure_path, today, 'features_cycle_liang')
        millis = int(round(time.time() * 1000))
        
        fig = plt.figure(figsize=(12, 10), dpi=300)
        plt.rcParams.update({'font.size': 16})
        plt.axis('off')
        plt.xlim((signal.index.min(), signal.index.max()))
        plt.scatter(signal.index[peaks], signal[peaks])
        plt.plot(signal.index, signal.values, linewidth=3.0, color="black")
        
        
    # Features
    window_df = pd.concat([window_df, pd.DataFrame({'start_ts': signal.index.min(),
                                               'sys_peak_ts': sys_peak_ts,
                                               'T_S': (sys_peak_ts - signal.index.min()).total_seconds(),
                                               'T_D': (signal.index.max() - sys_peak_ts).total_seconds()
                                              }, index=[cur_index])], sort=False)
    for p in [10, 25, 33, 50, 66, 75]:
        p_ampl = p / 100 * max_amplitude
        x1, x2 = find_xs_for_y(signal, p_ampl, peaks[0])
        if verbose:
            plt.scatter([x1, x2], signal[[x1, x2]])
        window_df.loc[cur_index, 'DW_'+str(p)] = (x2 - sys_peak_ts).total_seconds()
        window_df.loc[cur_index, 'DW_SW_sum_'+str(p)] = (x2 - x1).total_seconds()
        window_df.loc[cur_index, 'DW_SW_ratio_'+str(p)] = (x2 - sys_peak_ts) / (sys_peak_ts - x1)
    if verbose:
        plt.show()
        if not os.path.exists(features_cycle):
            os.makedirs(features_cycle)
        fig.savefig(os.path.join(features_cycle, str(millis)+'_features_cycle_liang.svg'), 
                    transparent=True, format='svg')
    return window_df
    
def extract_features_for_window(df, verbose=False):
    cycles = find_clean_cycles_with_template(df['bvp_filtered'], verbose=verbose)
    # All signal peaks
    all_peaks = sig.find_peaks(df['bvp_filtered'].values, distance=0.7*1000)[0]
    if verbose:
        print('All Peaks: ', all_peaks)
    if len(cycles) == 0:
        return pd.DataFrame()
    
    window_features = pd.DataFrame()
    cur_index = 0
    for i in range(len(cycles)):
        window_features = extract_features_for_cycle(window_features, df['bvp_filtered'].iloc[cycles[i][0]:cycles[i][1]], verbose=verbose)
        if i > 0:
            window_features.loc[cur_index-1, 'CP'] = (window_features.loc[cur_index, 'sys_peak_ts'] - window_features.loc[cur_index-1, 'sys_peak_ts']).total_seconds()
        cur_index = cur_index + 1
    # Last cycle or only cycle need to relies on the difference between the general peaks
    all_peaks_index = len(all_peaks)-1
    window_features.loc[cur_index-1, 'CP'] = (all_peaks[all_peaks_index] - all_peaks[all_peaks_index-1])/1000

    if verbose:
        print('Cycle Features within Window:')
        print(window_features)
   
    # Remove Outliers
    window_features = clean_window_features_of_outliers(window_features)
    
    if verbose:
        print('Cycle Features within Window no Outliers:')
        print(window_features)
    return window_features

def extract_features_for_signal(signal, subject_info, verbose=False):   
    window_features = extract_features_for_window(signal, verbose)
    if window_features.empty:
        return pd.DataFrame()
    for col in window_features.columns:
        if col.find('ts') == -1:
            subject_info[col+'_mean'] = window_features[col].mean()
            subject_info[col+'_var'] = window_features[col].var()
    subject_info.dropna(inplace=True, how='any')
    return subject_info

In [ ]:
def extract_features(csv=True, time_delta='2.1 sec', time_delta_type = 'bfill', 
                     experiment_type='liang', motion_filter=False, special_filter='cheby', verbose=False):
    
    with open('../../config.json') as f:
        config = json.load(f)

    exp_base_path = config['liang']
    print(exp_base_path)
    figure_path = config['figures']
    
    if verbose:
        print(exp_base_path)
        print(figure_path)
        print('\n')

    today = datetime.datetime.today().strftime('%Y-%m-%d')
    all_features = pd.DataFrame()
    subject_files = get_files(exp_base_path)

    # Create features path if it does not exist
    features_path = os.path.join('..', '..', 'features', 'liang', today, experiment_type.replace(' ',''), time_delta.replace(' ','')+'-'+time_delta_type+'-'+str(special_filter).lower(), 'motion-not-filtered')

    if not os.path.exists(features_path):
        os.makedirs(features_path)

    print('Extracting Features...')
    print('-----','\n')
    
    for subject_file in subject_files:
        info = subject_file.split('_')
        subject = info[0]
        segment = info[1][:-4]
        subject_file_name = os.path.join(exp_base_path, subject_file)
        
        print("Subjec: ", subject)
        print("Segment: ", segment)
        
        subject_data = pd.read_csv(subject_file_name, lineterminator='\t', header=None, names=['bvp'])
        if verbose:
            subject_data['bvp'].plot(figsize=(10, 10))
            plt.show()
        
        # Subject and Segment
        subject_info = pd.DataFrame(data={'subject': [subject], 'segment': [segment]})
        normalized_data = normalise(subject_data, 'min_max')
        
        if verbose:
            fig = plt.figure(figsize=(14, 6))
            plt.plot(subject_data.index, subject_data['bvp'])
            plt.xlabel('Samples')
            plt.ylabel('Normalized Magnitude')

            ppg_figure_path = os.path.join(figure_path, today, 'ppg_liang')
            if not os.path.exists(ppg_figure_path):
                os.makedirs(ppg_figure_path)
            
            fig.savefig(os.path.join(ppg_figure_path, 'normal_liang.pdf'), dpi=300, format='pdf')
            plt.show()
        
        # Apply filter, e. g. moving average, cheby, butter
        if special_filter:
            filtered_data = apply_filter(normalized_data, special_filter, verbose=verbose)
        else:
            if verbose: print("Not filtered.")
       
        features = extract_features_for_signal(filtered_data, subject_info, verbose=verbose)
        
        print('Features: ', features.shape)

        if features.empty:
            print('-----','\n')
            continue
            
        if csv:
            features.to_csv(features_path+'/features_{}_{}_{}_{}.csv'.format(subject, segment, experiment_type.replace(' ',''), time_delta.replace(' ','')+'-'+time_delta_type+'-'+str(special_filter).lower()), index=False)
        if verbose: 
            display(features)

        if all_features.empty:
            all_features = features
        else:
            all_features = pd.concat([all_features, features])
        print('-----','\n')
#         break
        
    all_features['time_delta'] = time_delta
    all_features['time_delta_type'] = time_delta_type
    all_features['experiment_type'] = experiment_type
    all_features['motion_filter'] = motion_filter
    all_features['special_filter'] = special_filter
    
    # Adding sorting by subject
    if 'subject' in all_features:
        all_features['subject'] = all_features['subject'].astype(int)
        all_features.sort_values(by=['subject','segment'], inplace=True, ignore_index=True)
        
    if csv:
        all_features_path = features_path+'/all_features_{}_{}.csv'.format(experiment_type.replace(' ',''), time_delta.replace(' ','')+'-'+time_delta_type+'-'+str(special_filter).lower())
        all_features.to_csv(all_features_path, index=False)
    
    print('Amount of BP-Pairs: ', all_features.shape)
    print('Features Extracted.')
    print('-----','\n')
    
    if csv:
        return all_features_path
    else:
        return all_features

# Extract Features

In [ ]:
feature_parameters = {
                        'csv' : True,
                        'time_delta' : '2.1 sec',
                        'time_delta_type': 'ffill',
                        'experiment_type' : 'liang',
                        'special_filter' : 'cheby',
                        'verbose' : False
                        }

path = extract_features(csv=feature_parameters['csv'],
                        time_delta=feature_parameters['time_delta'], 
                        time_delta_type=feature_parameters['time_delta_type'], 
                        experiment_type = feature_parameters['experiment_type'], 
                        special_filter = feature_parameters['special_filter'], 
                        verbose=feature_parameters['verbose'])
print(path)

# > The following experiments were not included in the JAIME 2021 paper, but feel free to play around!

# Read All Paths

In [ ]:
time_delta = '2.1 sec'
time_delta_type = 'ffill'
experiment_type = 'liang'
special_filters = ['cheby']

all_paths = []
date = datetime.datetime.today().strftime('%Y-%m-%d')

for special_filter in special_filters:
    features_path = os.path.join('..', '..', 'features', 'liang', date, experiment_type.replace(' ',''), time_delta.replace(' ','')+'-'+time_delta_type+'-'+str(special_filter).lower())
    motion_path = os.path.join(features_path, 'motion-not-filtered')
    path = os.path.join(motion_path,'all_features_{}_{}.csv'.format(experiment_type.replace(' ',''), time_delta.replace(' ','')+'-'+time_delta_type+'-'+str(special_filter).lower()))
    all_paths.append(path)
print(all_paths)

# Join BP Values and Features

In [ ]:
with open('../../config.json') as f:
        config = json.load(f)
        
experiments = config['liang']
patient_info = os.path.join(experiments[:-5], 'PPG-BP dataset.xlsx')

print(patient_info)
patients = pd.read_excel(patient_info, skiprows=1, engine='openpyxl',).rename(
    columns={"Systolic Blood Pressure(mmHg)":'SYS(mmHg)', 'subject_ID':'subject',
            "Diastolic Blood Pressure(mmHg)":'DIA(mmHg)',"Hypertension" : "class"})
patients = patients.replace({'class': 
                             {'Stage 2 hypertension' : 1.0, 'Stage 1 hypertension': 1.0, 
                              'Prehypertension' : 0.0, 'Normal' : 0.0}})
patient_subset = patients[['SYS(mmHg)','DIA(mmHg)','subject','class']]
patient_subset.head()

In [ ]:
patient_subset.groupby('class').count()

In [ ]:
df = pd.read_csv(all_paths[0])
df.head(2)

# Drop Lines with Empty Columns

In [ ]:
df.dropna(axis=0, how='any', inplace=True)
df.shape

In [ ]:
new_path = all_paths[0][:-len(all_paths[0].split('/').pop())]

patients = patient_subset.join(df.set_index('subject'), on='subject', how='inner')
print(patients.shape)
display(patients.head(5))
patients.to_csv(new_path+'/all_patients_with_bp.csv', index=False)

# Experiments

In [ ]:
experiments = []

df = patients.copy()
print("Complete: ", df.shape)

df_hype = df.loc[patients['class'] == 1.0].copy()
df_hype.loc[:,'experiment_type'] = 'liang_hype'
print("Hype: ", df_hype.shape)

df_normo = df.loc[patients['class'] == 0.0].copy()
df_normo.loc[:,'experiment_type'] = 'liang_normo'
print("Normo: ", df_normo.shape)

if not df.empty:
        predicted_variables = ['SBP', 'DBP']
        ks = [1]
        
        features = {
                    'time_delta' : df['time_delta'].unique()[0], 
                    'time_delta_type' : df['time_delta_type'].unique()[0],
                    'experiment_type' : df['experiment_type'].unique()[0], 
                    'motion_filter' : df['motion_filter'].unique()[0],
                    'special_filter' : df['special_filter'].unique()[0],
                    'segment' : df['segment'].unique()[0],
                    'class' : df['segment'].unique()[0]
                    }
        
        df.drop(features.keys(), axis=1, inplace=True)
        
        features_hype = {
                    'time_delta' : df_hype['time_delta'].unique()[0], 
                    'time_delta_type' : df_hype['time_delta_type'].unique()[0],
                    'experiment_type' : df_hype['experiment_type'].unique()[0], 
                    'motion_filter' : df_hype['motion_filter'].unique()[0],
                    'special_filter' : df_hype['special_filter'].unique()[0],
                    'segment' : df_hype['segment'].unique()[0],
                    'class' : df_hype['segment'].unique()[0]
                    }
        
        df_hype.drop(features_hype.keys(), axis=1, inplace=True)
        
        features_normo = {
                    'time_delta' : df_normo['time_delta'].unique()[0], 
                    'time_delta_type' : df_normo['time_delta_type'].unique()[0],
                    'experiment_type' : df_normo['experiment_type'].unique()[0], 
                    'motion_filter' : df_normo['motion_filter'].unique()[0],
                    'special_filter' : df_normo['special_filter'].unique()[0],
                    'segment' : df_normo['segment'].unique()[0],
                    'class' : df_normo['segment'].unique()[0]
                    }
        
        df_normo.drop(features_normo.keys(), axis=1, inplace=True)
       
        for variable in predicted_variables:
            for k in ks:
                df_total = df
                results = predict_bp_from_ppg(df_total, predicted_variable=variable, k=k, 
                                              correlation_threshold = 0.9, verbose=False)
                results.update(features)
                experiments.append(results)
                
                # Hypertensive
                results_hype = predict_bp_from_ppg(df_hype, predicted_variable=variable, k=k, 
                                              correlation_threshold = 0.6, verbose=False)
                results_hype.update(features_hype)
                experiments.append(results_hype)
                
                # Normotensive + Prehype
                results_normo = predict_bp_from_ppg(df_normo, predicted_variable=variable, k=k, 
                                              correlation_threshold = 0.75, verbose=False)
                results_normo.update(features_normo)
                experiments.append(results_normo)
                
else:
    ("Dataframe was empty: ", path)
all_experiments = pd.DataFrame.from_dict(experiments)

# Saving the Results

In [ ]:
all_experiments = all_experiments.replace({'motion_filter': {True : 'yes', False: 'no'}})
all_experiments

In [ ]:
all_experiments[['experiment_type','predicted_variable','MAE_LR_MEAN','MAE_GBM_MEAN','MAE_LGBM_MEAN','MAE_RF_MEAN']]

In [ ]:
date = datetime.datetime.today().strftime('%Y-%m-%d')
total = all_experiments.loc[all_experiments['experiment_type'] == 'liang']
print(total.shape)
total.to_csv('../../results/'+date+'_results_liang.csv', index=True, mode='w')

## Read Experiments From Results File and Exploring the Data

In [ ]:
date = datetime.datetime.today().strftime('%Y-%m-%d')
results_path = '../../results/'+date+'_results_liang.csv'
all_experiments = pd.read_csv(results_path)
all_experiments

In [ ]:
gbm = all_experiments[['predicted_variable','MAE_GBM_MEAN','MAE_GBM_STD','MAPE_GBM_MEAN','RMSE_GBM_MEAN','k']].sort_values(by=['predicted_variable','k','MAE_GBM_MEAN'])
lr = all_experiments[['predicted_variable','MAE_LR_MEAN','MAE_LR_STD','MAPE_LR_MEAN','RMSE_LR_MEAN','k']].sort_values(by=['predicted_variable','k','MAE_LR_MEAN'])
lgbm = all_experiments[['predicted_variable','MAE_LGBM_MEAN','MAE_LGBM_STD','MAPE_LGBM_MEAN','RMSE_LGBM_MEAN','k']].sort_values(by=['predicted_variable','k','MAE_LGBM_MEAN'])
rf = all_experiments[['predicted_variable','MAE_RF_MEAN','MAE_RF_STD','MAPE_RF_MEAN','RMSE_RF_MEAN','k']].sort_values(by=['predicted_variable','k','MAE_RF_MEAN'])
d = all_experiments[['predicted_variable','experiment_type','MAE_DUMMY_MEAN','MAE_DUMMY_STD','MAPE_DUMMY_MEAN','RMSE_DUMMY_MEAN','special_filter','time_delta','time_delta_type','motion_filter','k']].sort_values(by=['predicted_variable','experiment_type','k','MAE_DUMMY_MEAN'])

## Best Results

In [ ]:
# min liang dbp
gbm_min_liang_dbp = gbm.loc[(gbm['predicted_variable'] == 'DBP') & (gbm['k'] == 1)].head(1)[
    ['predicted_variable','k','MAE_GBM_MEAN','MAE_GBM_STD']]
lgbm_min_liang_dbp = lgbm.loc[(lgbm['predicted_variable'] == 'DBP') & (lgbm['k'] == 1)].head(1)[
    ['predicted_variable','k','MAE_LGBM_MEAN','MAE_LGBM_STD']]
rf_min_liang_dbp = rf.loc[(rf['predicted_variable'] == 'DBP') & (rf['k'] == 1)].head(1)[
    ['predicted_variable','k','MAE_RF_MEAN','MAE_RF_STD']]
lr_min_liang_dbp = lr.loc[(lr['predicted_variable'] == 'DBP') & (lr['k'] == 1)].head(1)[
    ['predicted_variable','k','MAE_LR_MEAN','MAE_LR_STD']]
dummy_min_liang_dbp = d.loc[(d['predicted_variable'] == 'DBP') & (d['k'] == 1)].head(1)[
    ['predicted_variable','k','MAE_DUMMY_MEAN','MAE_DUMMY_STD']]

min_liang_dbp = gbm_min_liang_dbp.set_index(['predicted_variable','k']).join(lgbm_min_liang_dbp.set_index(
    ['predicted_variable','k']), on=['predicted_variable','k'])
min_liang_dbp = min_liang_dbp.join(rf_min_liang_dbp.set_index(['predicted_variable','k']), 
                                   on=['predicted_variable','k'])
min_liang_dbp = min_liang_dbp.join(lr_min_liang_dbp.set_index(['predicted_variable','k']), 
                                   on=['predicted_variable','k'])
min_liang_dbp = min_liang_dbp.join(dummy_min_liang_dbp.set_index(['predicted_variable','k']), 
                                   on=['predicted_variable','k'])

# min liang sbp
gbm_min_liang_sbp = gbm.loc[(gbm['predicted_variable'] == 'SBP') & (gbm['k'] == 1)].head(1)[
    ['predicted_variable','k','MAE_GBM_MEAN','MAE_GBM_STD']]
lgbm_min_liang_sbp = lgbm.loc[(lgbm['predicted_variable'] == 'SBP') & (lgbm['k'] == 1)].head(1)[
    ['predicted_variable','k','MAE_LGBM_MEAN','MAE_LGBM_STD']]
rf_min_liang_sbp = rf.loc[(rf['predicted_variable'] == 'SBP') & (rf['k'] == 1)].head(1)[
    ['predicted_variable','k','MAE_RF_MEAN','MAE_RF_STD']]
lr_min_liang_sbp = lr.loc[(lr['predicted_variable'] == 'SBP') & (lr['k'] == 1)].head(1)[
    ['predicted_variable','k','MAE_LR_MEAN','MAE_LR_STD']]
dummy_min_liang_sbp = d.loc[(d['predicted_variable'] == 'SBP') & (d['k'] == 1)].head(1)[
    ['predicted_variable','k','MAE_DUMMY_MEAN','MAE_DUMMY_STD']]

min_liang_sbp = gbm_min_liang_sbp.set_index(['predicted_variable','k']).join(lgbm_min_liang_sbp.set_index(
    ['predicted_variable','k']), on=['predicted_variable','k'])
min_liang_sbp = min_liang_sbp.join(rf_min_liang_sbp.set_index(['predicted_variable','k']), 
                                   on=['predicted_variable','k'])
min_liang_sbp = min_liang_sbp.join(lr_min_liang_sbp.set_index(['predicted_variable','k']), 
                                   on=['predicted_variable','k'])
min_liang_sbp = min_liang_sbp.join(dummy_min_liang_sbp.set_index(['predicted_variable','k']), 
                                   on=['predicted_variable','k'])
min_liang_sbp

best_results = pd.concat([min_liang_dbp, min_liang_sbp], axis=0)
best_results[['MAE_GBM_MEAN','MAE_GBM_STD','MAE_LGBM_MEAN','MAE_LGBM_STD',
              'MAE_LR_MEAN','MAE_LR_STD','MAE_DUMMY_MEAN','MAE_DUMMY_STD']]

In [ ]:
date = datetime.datetime.today().strftime('%Y-%m-%d')
best_results.to_csv('../../results/'+date+'_best_results_liang.csv', index=True, mode='w')